<a href="https://colab.research.google.com/github/DenisR144/healthdata22/blob/main/Dataton_2022_(DR_and_T_SH).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("aa")